<a href="https://colab.research.google.com/github/marinavasqr23/HW1-ICA/blob/main/Homework_1_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ANÁLISE DE DADOS DATASET "WHINE QUALITY"

# 1. IMPORTANDO BIBLIOTECAS

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import seaborn as sns
import math



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 2. TRATANDO OS DADOS

In [ ]:
import pandas as pd

# Verify the contents of the directory
!ls "/content/drive/MyDrive/Homework/"

# Load the datasets
dt_whine_red = pd.read_csv("/content/drive/MyDrive/Homework/winequality-red.csv", sep=";")
dt_whine_white = pd.read_csv("/content/drive/MyDrive/Homework/winequality-white.csv", sep=";")

In [ ]:
dt_whine_red.info()

In [ ]:
dt_whine_white.info()

In [ ]:
# CONFERINDO INCONSISTÊNCIA ENTRE OS VALORES PARA PADRONIZAR OS TIPOS
# Tudo OK
dt_whine_white['alcohol'].unique()

In [ ]:
# CONFERINDO INCONSISTÊNCIA ENTRE OS VALORES PARA PADRONIZAR OS TIPOS
# Observe que alguns valores estranhos e por isso a coluna está como Object
dt_whine_red['alcohol'].unique()

In [ ]:
# Lista com os valores a serem corrigidos e suas substituições
dict_whine_red = {
    '100.333.333.333.333': '100',
    '110.666.666.666.667': '110',
    '956.666.666.666.667': '956',
    '135.666.666.666.667': '135',
    '923.333.333.333.333': '923'
}

In [ ]:
# Substituindo os valores
dt_whine_red['alcohol'] = dt_whine_red['alcohol'].replace(dict_whine_red)

In [ ]:
# Valores substituídos
dt_whine_red['alcohol'].unique()

In [ ]:
# Alterando para numeric
dt_whine_red['alcohol'] = pd.to_numeric(dt_whine_red['alcohol'])

In [ ]:
# Dividindo os  valores substituídos
dt_whine_red['alcohol'] = np.where(
    dt_whine_red['alcohol'] >= 100,
    dt_whine_red['alcohol'] / 100,
    dt_whine_red['alcohol']
)

In [ ]:
dt_whine_red['alcohol'].unique()

In [ ]:
dt_whine_red.info()

In [ ]:
# Fazemos um concatenção da tabelas
dt_whine_all = pd.concat([dt_whine_red, dt_whine_white], ignore_index=True)

In [ ]:
# Analisando nosso dataset percebemos uma discrepancia entre o minimo e maximo da densidade
dt_whine_all.describe()

In [ ]:
# Analisando valores maiores que média da densidade, observamos que temos 4 linhas com valores muito discrepantes
bigger_than_one = dt_whine_red[dt_whine_red['density'] > 1]['density']
print("Valores > 1:")
print(bigger_than_one.head())

In [ ]:
dt_whine_red['density'] = np.where (
    dt_whine_red['density'] > 1,
    dt_whine_red['density'] > 100,
    dt_whine_red['density']
)

# 3. ANÁLISE UNIVARIADA INCONDICIONAL

3.1. Histograma dos Preditores

In [ ]:
def univariate_statistics_histogram_frequency(dataset):
    predictors_columns = dataset.columns[:11]

    fig, axs = plt.subplots(4, 3, figsize=(10, 12))
    axs = axs.flatten()

    colors = ['#FFB6C1', '#87CEFA', '#98FB98', '#DDA0DD', '#FFFACD','#F0E68C', '#E6E6FA', '#F5DEB3', '#B0E0E6', '#FFDAB9', '#DA70D6']

    for i, column in enumerate(predictors_columns):
        data = dataset[column]

        axs[i].hist(data, bins=20, color=colors[i], edgecolor='black', alpha=0.7)
        axs[i].set_title(f"{column.title()}")
        axs[i].set_xlabel(column.title())
        axs[i].set_ylabel("Frequência")
        axs[i].grid(True, alpha=0.3)

    for i in range(len(predictors_columns), len(axs)):
        axs[i].set_visible(False)

    plt.tight_layout()
    plt.show()
univariate_statistics_histogram_frequency(dt_whine_red)

3.2. Calculando os valores da Média, Desvio Padrão e Assimetria de cada preditor

In [ ]:
def univariate_statistics_values(dt_whine_all):
    values = []
    predictors_columns = dt_whine_all.columns[:11]

    for column in predictors_columns:
        mean = dt_whine_all[column].mean() # Média
        std_dev = dt_whine_all[column].std() # Desvio padrão
        skewness = dt_whine_all[column].skew() # Assimetria

        values.append({
            "Preditor": column,
            "Média (µ)": round(mean, 4),
            "Desvio Padrão (σ)": round(std_dev, 4),
            "Assimetria (γ)": round(skewness, 4)
        })

    results = pd.DataFrame(values)
    return results

table_statistics = univariate_statistics_values(dt_whine_all)
print(table_statistics.to_string(index=False))

# 5. Análise Bivariada Incondicional

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

def bivariate_statistics_scatter(dataset):
    predictors_columns = dataset.columns[:11]  # seleciona os 11 preditores numéricos
    num_pairs = len(predictors_columns)

    # Gera pares de variáveis (só combinações únicas)
    pairs = [(predictors_columns[i], predictors_columns[j])
             for i in range(num_pairs) for j in range(i + 1, num_pairs)]

    # Seleciona até 12 pares para visualizar (para não ficar enorme)
    pairs = pairs[:12]

    fig, axs = plt.subplots(4, 3, figsize=(12, 12))
    axs = axs.flatten()

    colors = ['#FFB6C1', '#87CEFA', '#98FB98', '#DDA0DD', '#FFFACD',
              '#F0E68C', '#E6E6FA', '#F5DEB3', '#B0E0E6', '#FFDAB9', '#DA70D6', '#FFC0CB', '#B0C4DE']

    for i, (x, y) in enumerate(pairs):
        sns.scatterplot(data=dataset, x=x, y=y, alpha=0.5, color=colors[i], edgecolor=None, ax=axs[i])
        axs[i].set_title(f"{x.title()} vs {y.title()}", fontsize=10)
        axs[i].set_xlabel(x.title())
        axs[i].set_ylabel(y.title())
        axs[i].grid(True, alpha=0.3)

    for i in range(len(pairs), len(axs)):
        axs[i].set_visible(False)

    plt.tight_layout()
    plt.show()

# Exemplo de uso:
bivariate_statistics_scatter(dt_whine_all)

# 6. Análise Multivariada Incondicional

In [ ]:
# Implement PCA from scratch, attempt to load referenced winequality files if present; otherwise fall back to Iris dataset.
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Helper: manual PCA implementation
def manual_pca(X, n_components=2, standardize=True):
    # X: numpy array (n_samples, n_features)
    if standardize:
        mean = np.mean(X, axis=0)
        std = np.std(X, axis=0, ddof=1)
        std_repl = std.copy()
        std_repl[std_repl == 0] = 1.0
        Xs = (X - mean) / std_repl
    else:
        Xs = X - np.mean(X, axis=0)
    # Covariance matrix (features x features)
    cov = np.cov(Xs, rowvar=False, ddof=1)
    # Eigen decomposition
    eigvals, eigvecs = np.linalg.eigh(cov)  # eigh for symmetric
    # Sort eigenvalues descending
    idx = np.argsort(eigvals)[::-1]
    eigvals = eigvals[idx]
    eigvecs = eigvecs[:, idx]
    # Select top components
    components = eigvecs[:, :n_components]
    # Project data
    projected = Xs.dot(components)
    # Explained variance ratio
    explained_variance_ratio = eigvals / np.sum(eigvals)
    return {
        "mean": mean,
        "std": std_repl if standardize else None,
        "components": components,
        "eigvals": eigvals,
        "explained_variance_ratio": explained_variance_ratio,
        "projected": projected
    }

# Try to load winequality datasets referenced in the uploaded script
paths = [
    "/content/drive/MyDrive/Homework/winequality-red.csv",
    "/content/drive/MyDrive/Homework/winequality-white.csv",
    "/mnt/data/winequality-red.csv",
    "/mnt/data/winequality-white.csv",
    "./winequality-red.csv",
    "./winequality-white.csv"
]
wine_files = [p for p in paths if os.path.exists(p)]
data_used = None

if len(wine_files) >= 2:
    red = pd.read_csv(wine_files[0], sep=";")
    white = pd.read_csv(wine_files[1], sep=";")
    df = pd.concat([red, white], ignore_index=True)
    # create class label if not present: use 'quality' as class (binned)
    if 'quality' in df.columns:
        labels = df['quality'].astype(str).values
    else:
        labels = np.zeros(len(df), dtype=str)
    X = df.select_dtypes(include=[np.number]).values
    data_used = "wine (from local files)"
else:
    # Fall back to iris dataset
    try:
        from sklearn import datasets
        iris = datasets.load_iris()
        X = iris.data
        labels = iris.target
        target_names = iris.target_names
        data_used = "iris (sklearn)"
    except Exception as e:
        # If sklearn not available, create synthetic dataset
        rng = np.random.RandomState(0)
        X = np.vstack([rng.normal(loc=-2, scale=1, size=(50,4)),
                       rng.normal(loc=2, scale=1, size=(50,4)),
                       rng.normal(loc=5, scale=1, size=(50,4))])
        labels = np.array([0]*50 + [1]*50 + [2]*50)
        target_names = np.array(["class0","class1","class2"])
        data_used = "synthetic"

# Run PCA
result = manual_pca(X, n_components=2, standardize=True)
proj = result["projected"]
explained = result["explained_variance_ratio"]

# Plotting scatter with class labels as different markers/colors
unique_labels, inv = np.unique(labels, return_inverse=True)
markers = ['o', 's', '^', 'D', 'v', 'P', 'X']  # cycle markers
plt.figure(figsize=(8,6))
for i, lab in enumerate(unique_labels):
    mask = (inv == i)
    plt.scatter(proj[mask,0], proj[mask,1], label=str(lab), marker=markers[i % len(markers)], edgecolor='k', linewidth=0.4, s=60)
plt.xlabel("PC1 (%.2f%% variance)" % (explained[0]*100))
plt.ylabel("PC2 (%.2f%% variance)" % (explained[1]*100))
plt.title("PCA projection onto first two principal components — %s" % data_used)
plt.legend(title="Class", bbox_to_anchor=(1.05,1), loc='upper left')
plt.grid(True)
plt.tight_layout()
plt.show()

# Print summary (explained variance ratios)
print("Data used:", data_used)
print("Explained variance ratios (top 5):", np.round(explained[:5], 4))
print("Eigenvalues (top 5):", np.round(result['eigvals'][:5], 4))
